| [![License: MIT](https://img.shields.io/badge/License-MIT-green.svg)](../LICENSE) | [![Python](https://img.shields.io/badge/Python-3.10+-black.svg)](https://www.python.org/) | [![Jupyter](https://img.shields.io/badge/Jupyter-Notebook-red.svg)](https://jupyter.org/) | [![Pandas](https://img.shields.io/badge/Data-Pandas-purple.svg)](https://pandas.pydata.org/) | [![Matplotlib](https://img.shields.io/badge/Plots-Matplotlib-darkgreen.svg)](https://matplotlib.org/) | [![Seaborn](https://img.shields.io/badge/Plots-Seaborn-teal.svg)](https://seaborn.pydata.org/) | [![SciPy](https://img.shields.io/badge/Stats-SciPy-navy.svg)](https://scipy.org/) |
|---|---|---|---|---|---|---|


## Notebook 2 – Exploratory Analysis and Quality Checks  

**LuftDataQC: PM2.5 analysis from NILU API – Skøyen and Furulund (2023)**  
Source: [https://api.nilu.no](https://api.nilu.no)  

---

### EN: Project overview  
**Goal:** Load and transform hourly PM2.5 data, apply exploratory data analysis (EDA), and run quality control checks to identify missing values, gaps, and outliers. Prepare clean datasets for reporting and modeling.  
**Method:** SQLite → pandas → EDA → visual checks → statistical tests → clean export.  
**Tools:** Python (`pandas`, `matplotlib`, `seaborn`, `sqlite3`). 

### NO: Prosjektoversikt  
**Mål:** Lese og transformere timesvise PM2.5-målinger, gjennomføre utforskende datanalyse (EDA) og utføre Kvalitetskontroll for å oppdage mangler og avvik. Klargjøre datasettet for videre bruk.  
**Metode:** SQLite → pandas → EDA → visuelle sjekker → statistiske tester → eksport.  
**Verktøy:** Python (`pandas`, `matplotlib`, `seaborn`, `sqlite3`). 

---

### Reproducibility — quick reference | Reproduserbarhet — hurtigoversikt  

**Inputs (from Notebook 1):**  
- `data/processed/pm25_2023.sqlite` *(tables: pm25_skøyen, pm25_furulund | tabeller: pm25_skøyen, pm25_furulund)*  

**Outputs (this notebook):**  
- `results/pm25_distribution_box_hist_skøyen_furulund.png` *(Comparative histograms & boxplots | Sammenlignende histogrammer og boksplott)*  
- `results/pm25_monthly_trends_skøyen_furulund.png` *(Seasonal/long-term monthly patterns | Sesongmessige/langsiktige månedsmønstre)*  
- `results/pm25_hourly_trends_skøyen_furulund.png` *(Diurnal variation with mean ± std bands | Døgnvariasjon med gj.snitt ± std-bånd)*  
- `results/pm25_weekday_vs_weekend.png` *(Weekday vs weekend differences | Ukedager vs helg forskjeller)*  
- `results/pm25_trend_rolling_mean_skøyen_furulund.png` *(30-day rolling mean trends | 30-dagers glidende gjennomsnittstrender)*  

**Parameters (this notebook):**  
- **Year** = 2023  
- **Pollutant** = PM2.5  
- **EDA methods** = histogram, boxplot, lineplot, rolling means  
- **Statistical test** = Mann–Whitney U  
- **Export format** = Clean CSVs 

In [ ]:
# (EN) Reproducibility parameters and paths
# (NO) Reproduserbarhetsparametere og stier

YEAR = "2023"         
COMPONENT = "PM2.5"   

from pathlib import Path

# Define project paths (relative to /notebooks)
PROJECT_ROOT = Path.cwd().parent
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
RESULT_DIR = PROJECT_ROOT / "results"

# Ensure output folders exist
for folder in [RAW_DIR, PROCESSED_DIR, RESULT_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

# Define database path (same as in Notebook 1)
DB_PATH = PROCESSED_DIR / f"pm25_{YEAR}.sqlite"

# Define specific output file for this notebook
DISTRIBUTION_PNG = RESULT_DIR / f"pm25_distribution_skøyen_furulund.png"
MONTHLY_PNG = RESULT_DIR / "pm25_monthly_trends_skøyen_furulund.png"
HOURLY_PNG = RESULT_DIR / "pm25_hourly_trends_skøyen_furulund.png"
WEEKEND_PNG = RESULT_DIR /"pm25_weekday_vs_weekend.png"
ROLLING_PNG = RESULT_DIR /"pm25_trend_rolling_mean_skøyen_furulund.png"

In [ ]:
# (EN) Import required libraries and verify data/result folders
# (NO) Importerer nødvendige biblioteker og bekrefter mapper for data/resultater

# Standard library
import sqlite3

# Third-party libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu

# Confirm that folders exist
print(f"  • RAW_DIR exists:        {RAW_DIR.exists()}")
print(f"  • PROCESSED_DIR exists:  {PROCESSED_DIR.exists()}")
print(f"  • RESULT_DIR exists:     {RESULT_DIR.exists()}")


> **Note:** This project uses SQLite as a lightweight relational database to simulate real-world data storage and querying workflows.  
> In a production setting, this structure could be scaled to cloud platforms such as **Azure SQL**, **PostgreSQL**, or **Oracle** to support shared access, automation, and more complex integrations.



### Data Loading | Datainnhenting

**(EN)** The PM2.5 data is stored in a local **SQLite database**.Here, we connect to the database and load the relevant station tables into **pandas DataFrames**. From this point on, all analysis is performed on in-memory DataFrames (copies of the database tables), so the original database remains unchanged.

**(NO)** PM2.5-dataene er lagret i en lokal **SQLite-database**. Her kobler vi til databasen og laster inn de relevante stasjons-tabellene som **pandas DataFrames**. Fra dette punktet gjøres all analyse på DataFrames i minnet (kopier av databasetabellene), 
slik at den opprinnelige databasen forblir uendret.

In [ ]:
# (EN) Load hourly PM2.5 data from the SQLite database
# (NO) Last inn timesvise PM2.5-data fra SQLite-databasen

with sqlite3.connect(DB_PATH) as conn:
    skøyen_df   = pd.read_sql("SELECT * FROM pm25_skøyen", conn)
    furulund_df = pd.read_sql("SELECT * FROM pm25_furulund", conn)

# Show first rows to confirm
display(skøyen_df.head())
display(furulund_df.head())

In [ ]:
# (EN) Convert 'fromTime' column to datetime format
# (NO) Konverter 'fromTime' kolonnen til datetime-format

skøyen_df['fromTime'] = pd.to_datetime(skøyen_df['fromTime'])
furulund_df['fromTime'] = pd.to_datetime(furulund_df['fromTime'])

# Create datetime features: year, month, day, hour, weekday, and full month name

from pandas.api.types import CategoricalDtype

# Define correct calendar order for month names
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
month_cat = CategoricalDtype(categories=month_order, ordered=True)

# Extract features for both dataframes
for df in [skøyen_df, furulund_df]:
    df['year'] = df['fromTime'].dt.year
    df['month'] = df['fromTime'].dt.month
    df['day'] = df['fromTime'].dt.day
    df['hour'] = df['fromTime'].dt.hour
    df['weekday'] = df['fromTime'].dt.day_name()
    df['month_name'] = df['fromTime'].dt.month_name()
    df['month_name'] = df['month_name'].astype(month_cat)

# Check updated columns for Skøyen
display(skøyen_df[['fromTime', 'value', 'month', 'month_name', 'hour', 'weekday']].head())

# Check updated columns for Furulund
display(furulund_df[['fromTime', 'value', 'month', 'month_name', 'hour', 'weekday']].head())

In [ ]:
# (EN) Create a helper function to summarize data quality per station
# (NO) Lag en hjelpefunksjon for å oppsummere datakvalitet per stasjon

def quality_summary(df, name):
    print(f"\nStation: {name}")
    print("-" * 50)
    print(f"Total rows:              {len(df):,}")
    print(f"Date range:              {df['fromTime'].min()} → {df['fromTime'].max()}")
    print(f"Missing 'value':         {df['value'].isna().sum():,}")
    print(f"Duplicated timestamps:   {df['fromTime'].duplicated().sum():,}")
    print(f"Quality controlled:      {df['qualityControlled'].sum():,}")
    print(f"Not quality controlled:  {len(df) - df['qualityControlled'].sum():,}")
    print("-" * 50)

# Run summary for both stations
quality_summary(skøyen_df, "Skøyen")
quality_summary(furulund_df, "Furulund")

### Data Quality Overview | Oversikt over datakvalitet

##### **(EN)** This section presents a basic quality check of each station’s dataset.
##### **(NO)** Denne delen gir en grunnleggende kvalitetskontroll av datasettet for hver stasjon. 

In [ ]:
# (EN) Compute descriptive statistics for PM2.5 values at both stations
# (NO) Beregn beskrivende statistikk for PM2.5-verdier ved begge stasjoner

# Calculate summary statistics
desc_skøyen = skøyen_df['value'].describe() # Skøyen station
desc_furulund = furulund_df['value'].describe() # Furulund station

# Combine and round results for comparison
summary_df = pd.DataFrame({
    'Skøyen': desc_skøyen.round(2),
    'Furulund': desc_furulund.round(2)
})

# Display side by side
display(summary_df)

### Summary Statistics | Beskrivende statistikk

In [ ]:
# (EN) Create side-by-side plots (boxplot + histogram)
# (NO) Lag side-by-side figurer (boxplot + histogram)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Boxplots: visualize distribution and detect outliers

sns.boxplot(data=skøyen_df, y='value', ax=axes[0, 0], color='lightcoral')
axes[0, 0].set_title('Skøyen – Boxplot')
axes[0, 0].set_ylabel('PM2.5 (µg/m³)')

sns.boxplot(data=furulund_df, y='value', ax=axes[0, 1], color='lightblue')
axes[0, 1].set_title('Furulund – Boxplot')
axes[0, 1].set_ylabel('')

# Histograms: visualize distribution shape and density (with KDE)

sns.histplot(skøyen_df['value'], bins=50, ax=axes[1, 0], color='lightcoral', kde=True)
axes[1, 0].set_title('Skøyen – Histogram')
axes[1, 0].set_xlabel('PM2.5 (µg/m³)')
axes[1, 0].set_ylabel('Frequency')

sns.histplot(furulund_df['value'], bins=50, ax=axes[1, 1], color='lightblue', kde=True)
axes[1, 1].set_title('Furulund – Histogram')
axes[1, 1].set_xlabel('PM2.5 (µg/m³)')
axes[1, 1].set_ylabel('Frequency')

# Shared title
fig.suptitle("PM2.5 Distribution – Skøyen vs. Furulund (2023)", fontsize=16, y=1.02)

# Save figure to results folder and show plot 
DISTRIBUTION_PNG = RESULT_DIR / "pm25_distribution_box_hist_skøyen_furulund.png"
plt.tight_layout()
plt.savefig(DISTRIBUTION_PNG, dpi=300)
plt.show()

# Print confirmation 
print("Figure saved to:")
print(DISTRIBUTION_PNG)

>  **Note:** This figure is displayed and interpreted in the final report notebook (`04_report.ipynb`) 
>  
> **Merk:** Denne figuren vises og tolkes i den endelige rapportnotatboken (`04_report.ipynb`) 


### Statistical Test – Mann–Whitney U | Statistisk test – Mann–Whitney U

**(EN)** To determine whether the PM2.5 distributions for Skøyen and Furulund differ significantly, we apply the **Mann–Whitney U test**, a non-parametric alternative to the t-test. 

**(NO)** For å vurdere om PM2.5-fordelingene for Skøyen og Furulund er signifikant forskjellige, bruker vi **Mann–Whitney U-testen**, et ikke-parametrisk alternativ til t-testen. 


In [ ]:
# (EN) Statistical test to compare distributions of PM2.5 between Skøyen and Furulund
# (NO) Statistisk test for å sammenligne PM2.5-fordelinger mellom Skøyen og Furulund

# Use the same cleaned DataFrames used in previous visualizations
skøyen_values = skøyen_df['value']
furulund_values = furulund_df['value']

# Apply two-sided Mann–Whitney U test (non-parametric)
stat, p_value = mannwhitneyu(skøyen_values, furulund_values, alternative='two-sided')

# Print results
print("Mann–Whitney U test results:")
print(f" U-statistic: {stat:.2f}")
print(f" p-value:     {p_value:.4f}")

# Interpretation guide
if p_value < 0.05:
    print(" The difference in distributions is statistically significant (p < 0.05).")
else:
    print(" No statistically significant difference found (p ≥ 0.05).")

### Monthly PM2.5 Median (2023) – Skøyen vs. Furulund

In [ ]:
# (EN) Calculate monthly PM2.5 median, mean, and standard deviation, and plot seasonal trends
# (NO) Beregn månedlige verdier for median, gjennomsnitt og standardavvik av PM2.5, og visualiser sesongmønstre

# Group data by month and calculate mean and standard deviation
monthly_stats_skøyen = skøyen_df.groupby('month')['value'].agg(['median', 'mean', 'std'])
monthly_stats_furulund = furulund_df.groupby('month')['value'].agg(['median', 'mean', 'std'])

# Create plot of monthly medians with ±1 standard deviation bands

plt.figure(figsize=(12, 6))

# Skøyen: median line and standard deviation band
plt.plot(monthly_stats_skøyen.index, monthly_stats_skøyen['median'],
         label='Skøyen - Median', color='firebrick')
plt.fill_between(monthly_stats_skøyen.index,
                 monthly_stats_skøyen['median']- monthly_stats_skøyen['std'],
                 monthly_stats_skøyen['median'] + monthly_stats_skøyen['std'],
                 alpha=0.2, color='firebrick')

# Furulund: median line and standard deviation band
plt.plot(monthly_stats_furulund.index, monthly_stats_furulund['median'],
         label='Furulund – Median', color='steelblue')
plt.fill_between(monthly_stats_furulund.index,
                 monthly_stats_furulund['median'] - monthly_stats_furulund['std'],
                 monthly_stats_furulund['median'] + monthly_stats_furulund['std'],
                 alpha=0.2, color='steelblue')

# Final plot adjustments
plt.title("Monthly PM2.5 Median (2023) – Skøyen vs. Furulund")
plt.xlabel("Month")
plt.ylabel("PM2.5 (µg/m³)")
plt.legend()
plt.grid(True)
plt.tight_layout() 

# Save figure to results folder and show plot 
MONTHLY_PNG = RESULT_DIR / "pm25_monthly_trends_skøyen_furulund.png"
plt.tight_layout()
plt.savefig(MONTHLY_PNG, dpi=300)
plt.show()

# Print confirmation
print("Figure saved to:")
print(MONTHLY_PNG)

>  **Note:** This figure is displayed and interpreted in the final report notebook (`04_report.ipynb`). 
>  
> **Merk:** Denne figuren vises og tolkes i den endelige rapportnotatboken (`04_report.ipynb`).



### Hourly PM2.5 Mean (2023) – Skøyen vs. Furulund

In [ ]:
# (EN) Calculate hourly PM2.5 mean and standard deviation, and plot diurnal patterns
# (NO) Beregn gjennomsnitt og standardavvik for PM2.5 per time, og visualiser døgnmønstre

# Group data by hour and calculate mean and standard deviation
hourly_stats_skøyen = skøyen_df.groupby('hour')['value'].agg(['mean', 'std'])
hourly_stats_furulund = furulund_df.groupby('hour')['value'].agg(['mean', 'std'])

# Create plot of hourly means with ±1 standard deviation bands
plt.figure(figsize=(12, 6))

# Skøyen: mean line and standard deviation band
plt.plot(hourly_stats_skøyen.index, hourly_stats_skøyen['mean'], label='Skøyen – Mean', color='firebrick')
plt.fill_between(hourly_stats_skøyen.index,
                 hourly_stats_skøyen['mean'] - hourly_stats_skøyen['std'],
                 hourly_stats_skøyen['mean'] + hourly_stats_skøyen['std'],
                 alpha=0.2, color='firebrick')

# Furulund: mean line and standard deviation band
plt.plot(hourly_stats_furulund.index, hourly_stats_furulund['mean'], label='Furulund – Mean', color='steelblue')
plt.fill_between(hourly_stats_furulund.index,
                 hourly_stats_furulund['mean'] - hourly_stats_furulund['std'],
                 hourly_stats_furulund['mean'] + hourly_stats_furulund['std'],
                 alpha=0.2, color='steelblue')

# Final plot adjustments
plt.title("Hourly PM2.5 Mean (2023) – Skøyen vs. Furulund")
plt.xlabel("Hour of Day")
plt.ylabel("PM2.5 (µg/m³)")
plt.xticks(range(0, 24))
plt.grid(True)
plt.legend()
plt.tight_layout()

# Save figure to results folder and show plot 
HOURLY_PNG = RESULT_DIR / "pm25_hourly_trends_skøyen_furulund.png"
plt.tight_layout()
plt.savefig(HOURLY_PNG, dpi=300)
plt.show()

# Print confirmation
print("Figure saved to:")
print(HOURLY_PNG)

>  **Note:** This figure is displayed and interpreted in the final report notebook (`04_report.ipynb`).
>  
> **Merk:** Denne figuren vises og tolkes i den endelige rapportnotatboken (`04_report.ipynb`).


### Weekday vs. Weekend Comparison

In [ ]:
# (EN) Weekday vs. Weekend Comparison - PM2.5 Levels
# 
# This cell compares PM2.5 concentrations between weekdays and weekends for both stations.
# A boxplot is used to visually assess potential differences in pollution levels by day type.
#
# (NO) Ukedag vs. Helg - Sammenligning av PM2.5-nivåer
#
# Denne cellen sammenligner PM2.5-konsentrasjoner mellom ukedager og helger for begge stasjoner.
# Et boksplott brukes for å visuelt vurdere forskjeller i forurensningsnivå etter dagstype.

# Define whether each row is weekend or not
for df in [skøyen_df, furulund_df]:
    df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday'])

# Add station label to each dataset
skøyen_df['station'] = 'Skøyen'
furulund_df['station'] = 'Furulund'

# Combine both datasets into one DataFrame
combined = pd.concat([skøyen_df, furulund_df])

# Create comparative boxplot: PM2.5 vs. weekday/weekend
plt.figure(figsize=(12, 6))
sns.boxplot(data=combined, x='is_weekend', y='value', hue='station', palette=['firebrick', 'steelblue'])

plt.title("PM2.5 – Weekday vs. Weekend (2023)")
plt.xlabel("Is Weekend")
plt.ylabel("PM2.5 (µg/m³)")
plt.legend(title='Station')
plt.grid(True)
plt.tight_layout()

# Save figure to results folder
WEEKEND_PNG = RESULT_DIR / "pm25_weekday_vs_weekend.png"
plt.savefig(WEEKEND_PNG, dpi=300)
plt.show()

# Print confirmation
print("Figure saved to:")
print(WEEKEND_PNG)

>  **Note:** This figure is displayed and interpreted in the final report notebook (`04_report.ipynb`).
>  
> **Merk:** Denne figuren vises og tolkes i den endelige rapportnotatboken (`04_report.ipynb`).
>
> 



 ### Long-Term Trend Analysis – 30-Day Rolling Mean of PM2.5

**(EN)** A 30-day rolling mean is applied to highlight persistent seasonal pollution patterns and to smooth out short-term variability.

**(NO)** Et 30-dagers glidende gjennomsnitt brukes for å tydeliggjøre sesongmessige forurensningsmønstre og redusere kortsiktig variasjon.


In [ ]:
# (EN) Data handling practices: original DataFrames are used directly for descriptive analysis.
#     Copies are created only when transformations change their structure (e.g., resampling,
#     rolling, setting indexes).

# (NO) Databehandlingspraksis: de opprinnelige DataFrames brukes direkte til beskrivende analyser.
#     Kopier opprettes kun når transformasjoner endrer datastrukturen (f.eks. resampling,
#     rolling, sette indeks).

# Work on copies so originals stay intact
df_skøyen   = skøyen_df[['fromTime', 'value']].copy()
df_furulund = furulund_df[['fromTime', 'value']].copy()

# Step 1: Convert time column to datetime
df_skøyen["fromTime"] = pd.to_datetime(df_skøyen["fromTime"])
df_furulund["fromTime"] = pd.to_datetime(df_furulund["fromTime"])

# Step 2: Sort by time
df_skøyen = df_skøyen.sort_values("fromTime")
df_furulund = df_furulund.sort_values("fromTime")

# Step 3: Set datetime as index
df_skøyen.set_index("fromTime", inplace=True)
df_furulund.set_index("fromTime", inplace=True)

# Step 4: Calculate 30-day rolling mean (24×30 = 720 hours)
rolling_window = 24 * 30
df_skøyen["rolling_mean"] = df_skøyen["value"].rolling(rolling_window, min_periods=1).mean()
df_furulund["rolling_mean"] = df_furulund["value"].rolling(rolling_window, min_periods=1).mean()

# Step 5: Plot long-term trends
plt.figure(figsize=(14, 6))
plt.plot(df_skøyen.index, df_skøyen["rolling_mean"], label="Skøyen (Urban)", color="red")
plt.plot(df_furulund.index, df_furulund["rolling_mean"], label="Furulund (Residential)", color="green")

plt.title("30-day Rolling Mean of PM2.5")
plt.xlabel("Date | Dato")
plt.ylabel("PM2.5 (µg/m³)")
plt.legend()
plt.grid(True)

# Save figure to results folder
ROLLING_PNG = RESULT_DIR / "pm25_trend_rolling_mean_skøyen_furulund.png"
plt.tight_layout()
plt.savefig(ROLLING_PNG, dpi=300)
plt.show()

# Print confirmation
print("Figure saved to:")
print(ROLLING_PNG)

>  **Note:** This figure is displayed and interpreted in the final report notebook (`04_report.ipynb`). 
>  
> **Merk:** Denne figuren vises og tolkes i den endelige rapportnotatboken (`04_report.ipynb`).


#### Exploratory and Statistical Analysis Completed | Utforskende og statistisk analyse fullført

**EN:** This notebook produced descriptive statistics and key visualizations (distribution, hourly, monthly, weekday vs weekend, rolling trends) and justified the use of **Mann–Whitney U** due to non‑normal distributions. Outputs saved to `results/`.

**NO:** Denne notatboken genererte beskrivende statistikk og sentrale visualiseringer (fordeling, time/døgn, måned, ukedag vs helg, glidende trender) og begrunnet bruk av **Mann–Whitney U** grunnet ikke‑normal fordeling. Resultater lagret i `results/`.

_____

#### Next Step | Neste steg
Proceed to Notebook 3 to engineer time‑based features and apply **IsolationForest** for unsupervised anomaly detection.
_____

**Navigation Links**

- [Notebook 1 – Data Collection, Inspection and Storage](notebooks/01_data_sqlite.ipynb)  
- [Notebook 3 – Feature Engineering and Anomaly Detection](notebooks/03_features_anomalies.ipynb)  
- [Notebook 4 – Summary Report (EN)](notebooks/04_report.ipynb)  
- [Notebook 5 – Sammendragsrapport (NO)](notebooks/05_report_norsk.ipynb)

_____

**References note**
- Statistical rationale and implementation details are consolidated in **Notebook 4 - References** (Key sources: McDonald; Ghasemi and Zahediasl; SciPy `mannwhitneyu`.)


In [ ]:
# (EN) Notebook 2 complete
# (NO) Notebook 2 ferdig

print("Notebook 2 complete — outputs saved to results/, ready for Notebook 3 (Feature Engineering and Anomaly Detection).")
print("Notebook 2 ferdig — resultater lagret i results/, klar for Notebook 3 (Funksjonsutvikling og avviksdeteksjon).")